In [13]:
# Step 1: Create Synthetic Video Dataset (circle vs square)
import os
import cv2
import numpy as np

data_dir = '/content/synthetic_dataset'
os.makedirs(data_dir, exist_ok=True)

num_classes = 2
num_videos_per_class = 30   # zyada videos rakho for better accuracy
video_length = 30           # Number of frames
frame_height = 64
frame_width = 64

for class_id in range(num_classes):
    class_dir = os.path.join(data_dir, f'class_{class_id}')
    os.makedirs(class_dir, exist_ok=True)

    for video_id in range(num_videos_per_class):
        video_path = os.path.join(class_dir, f'video_{video_id}.avi')
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(video_path, fourcc, 20.0, (frame_width, frame_height))

        for _ in range(video_length):
            # Black background
            frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

            if class_id == 0:
                # Class 0 = circle
                cv2.circle(frame, (32, 32), 20, (0, 255, 0), -1)  # green circle
            else:
                # Class 1 = square
                cv2.rectangle(frame, (20, 20), (44, 44), (255, 0, 0), -1)  # blue square

            out.write(frame)

        out.release()

print("Synthetic video dataset (circle vs square) created!")

Synthetic video dataset (circle vs square) created!


In [14]:
# Step 2: Load the Synthetic Video Dataset
def load_videos(data_dir, img_size=(128, 128), max_frames=30):
    X, y = [], []
    class_names = os.listdir(data_dir)

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        for video_file in os.listdir(class_dir):
            video_path = os.path.join(class_dir, video_file)
            cap = cv2.VideoCapture(video_path)

            frames = []
            while len(frames) < max_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, img_size)
                frames.append(frame)

            if len(frames) == max_frames:
                X.append(frames)
                y.append(label)

            cap.release()

    return np.array(X), np.array(y), class_names

X, y, class_names = load_videos(data_dir)
X = X / 255.0  # Normalize the data

In [15]:
# Step 3: Split the Dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Step 4: Build the Model
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam # Import Adam optimizer

model = models.Sequential([
    layers.Conv3D(16, (3, 3, 3), activation='relu', input_shape=(30, 128, 128, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Dropout(0.5),

    layers.Conv3D(16, (3, 3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Dropout(0.5),

    layers.Conv3D(16, (3, 3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling3D((2, 2, 2)),
    layers.Dropout(0.5),

    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_names), activation='softmax')
])

# Instantiate the Adam optimizer with the desired learning rate
adam_optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=adam_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 5: Train the Model
model.fit(X_train, y_train, epochs=7, batch_size=32, validation_data=(X_test, y_test))

# Step 6: Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

# Step 7: Save the Model
model.save('video_model.h5')

Epoch 1/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.4903 - loss: 2.6647 - val_accuracy: 0.4643 - val_loss: 0.6769
Epoch 2/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 397ms/step - accuracy: 0.6674 - loss: 1.5144 - val_accuracy: 0.4643 - val_loss: 0.6696
Epoch 3/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 397ms/step - accuracy: 0.7816 - loss: 0.4989 - val_accuracy: 0.4643 - val_loss: 0.6674
Epoch 4/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 541ms/step - accuracy: 0.8309 - loss: 0.5252 - val_accuracy: 0.4643 - val_loss: 0.6603
Epoch 5/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 541ms/step - accuracy: 0.8544 - loss: 0.3871 - val_accuracy: 0.4643 - val_loss: 0.6483
Epoch 6/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 395ms/step - accuracy: 0.9488 - loss: 0.1467 - val_accuracy: 1.0000 - val_loss: 0.6414
Epoch 7/7
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 541ms/step - accuracy: 0.9548 - loss: 0.1120 - val_accuracy: 1.0000 - val_loss: 0.6390
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 1.0000 - loss: 0.6389


Test accuracy: 1.00
